# Building a Spam Filter with Naive Bayes

![](https://www.lifewire.com/thmb/DLAgQsq5sdhcl6lMAuR3nZJomYo=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/GettyImages-122143117-5c64996246e0fb0001f256b1.jpg)

Image Source: [Lifewire](https://www.lifewire.com/bayesian-spam-filtering-1164096)

## Table of Contents
---
- [Introduction](#Introduction)
- [Summary of Results](#Summary-of-Results)
- [Exploring the Dataset](#Exploring-the-Dataset)
- [Training and Test Set](#Training-and-Test-Set)
- [Letter Case and Punctuation](#Letter-Case-and-Punctuation)
- [Create the Vocabulary](#Create-the-Vocabulary)
- [The Final Training Set](#The-Final-Training-Set)
- [Calculating Constants First](#Calculating-Constants-First)
- [Calculating Parameters](#Calculating-Parameters)
- [Classifying A New Message](#Classifying-A-New-Message)
- [Measuring the Spam Filter's Accuracy](#Measuring-the-Spam-Filter's-Accuracy)
- [Conclusions](#Conclusions)

## Introduction
The aim of this project is to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm and a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from [this repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

To classify messages as spam or non-spam, the computer:

- Learns how humans classify messages.
- Uses that knowledge to estimate probabilities for new messages being spam or non-spam.
- Classifies a new message based on these values: 
     - the probability for spam is greater — spam,
     - the probability for non-spam is greater — non-spam,
     - the two probability values are equal — we may need a human to classify the message.
     
In other words, our task for this project is to "teach" the computer how to classify messages. 

### Summary of Results
We created a highly accurate spam filter, managing to reach the accuracy of 98.83%, which is almost 20% higher than our initial focus. A few messages classified incorrectly revealed some features in common. The attempt to increase the accuracy even further by making the algorithm sensitive to letter case resulted in just the opposite, rendering the spam filter 13.5% less efficient.

## Exploring the Dataset

We'll now start by reading in the dataset.

In [1]:
# Importing the necessary libraries 
import json
import pandas as pd

# Read the dataset
sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [2]:
sms.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


Let's now explore the dataset.

In [3]:
print(sms.shape)

(5572, 2)


In [4]:
# Finding the percentage of 'spam' and 'ham' messages in the dataset
sms['Label'].value_counts(normalize=True)*100 # normalize returns the relative frequency (dividing the number of times each of values appear by the sum of values)

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

We loaded the dataset and saw that about 86.6% of the messages are ham (i.e., non-spam), and the remaining 13.4% are spam. Now that we've become a bit more familiar with the dataset, we can move on to building the spam filter.

## Training and Test Set

Before creating our spam filter, let's think of a way to check how good it is with classifying new messages. We can do this by splitting our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing. The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

To better understand why we put a test set aside, let's observe that all 1,114 messages are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human. In this way, we'll see how good the spam filter is.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%. This means that we expect the spam filter to classify more than 80% of new messages correctly. Whether they are spam or non-spam. So now it's time to create a training and a test set. First off, let's randomize the entire dataset. This will ensure that spam and ham messages are spread properly. 

In [5]:
# Randomize the dataset
data_randomized = sms.sample(frac=1, random_state=123)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Training/Test split
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


Now that we've split the randomized dataset, let's calculate the percentage of spam and ham in both sets. In this way, we can check if these percentages are similar to what we have in the full dataset. This will then help us determine whether the sample is representative of the overall population. 

In [6]:
# Find percentage of labels in the training set
training_set['Label'].value_counts(normalize=True)*100

ham     86.608345
spam    13.391655
Name: Label, dtype: float64

In [7]:
# Find percentage of labels in the test set
test_set['Label'].value_counts(normalize=True)*100

ham     86.535009
spam    13.464991
Name: Label, dtype: float64

The results look good! We'll now move on to cleaning the dataset.

## Letter Case and Punctuation

Now that we've split our dataset into a training set and a test set, the next big step is to use the training set to teach the algorithm to classify new messages.

It's important to remember that, when a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations:


\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Moreover, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, it's important to mention that we will need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

For clarity, the terms in the equation above mean the following:

- N<sub>w<sub>i</sub></sub>|Spam=the number of times the word w<sub>i</sub> occurs in spam messages
- N<sub>w<sub>i</sub></sub>|Spam<sup>C</sup>=the number of times the word w<sub>i</sub> occurs in non-spam messages
- N<sub>Spam</sub>=total number of words in spam messages
- N<sub>Spam<sup>C</sup></sub>=total number of words in non-spam messages
- N<sub>Vocabulary</sub>=total number of words in the vocabulary
- α=1(α is a smoothing parameter)

Before going ahead and train our algorithm, we'll first need to clean the data. This will allow us to extract all the information we need more easily. Let's take another quick look at the dataset to remind ourselves of how its format looks like.

In [8]:
# Before cleaning
training_set.head(10)

,Label,SMS
0,ham,Aight text me when you're back at mu and I'll ...
1,ham,Our Prashanthettan's mother passed away last n...
2,ham,No it will reach by 9 only. She telling she wi...
3,ham,Do you know when the result.
4,spam,Hi. Customer Loyalty Offer:The NEW Nokia6650 M...
5,ham,It shall be fine. I have avalarr now. Will hol...
6,ham,I love u 2 my little pocy bell I am sorry but ...
7,ham,They finally came to fix the ceiling.
8,ham,S.i'm watching it in live..
9,spam,"Hi, the SEXYCHAT girls are waiting for you to ..."


As shown in the table above, the values under the `SMS` look quite messy and not easy to read. To make the data format more consistent, we will:
- Replace the `SMS` column by a series of new columns, where each column represents a unique word from the vocabulary.
- Each row will describe a single message, and all their values represent each word in the single message. Moreover, each column shows the frequency of that unique word for any given message.
- All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
- Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

Eventually, we will want our dataset to turn into the following format (the messages are fictitious to make the example easier to understand):

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)
                                                 
Let's start cleaning the data to see how this would look like in practice.

In [9]:
# Removing all punctuations from column SMS
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')

# Transforming every letter in every word to lower case from column SMS
training_set['SMS'] = training_set['SMS'].str.lower()

# After cleaning
training_set.head(10)

,Label,SMS
0,ham,aight text me when you re back at mu and i ll ...
1,ham,our prashanthettan s mother passed away last n...
2,ham,no it will reach by 9 only she telling she wi...
3,ham,do you know when the result
4,spam,hi customer loyalty offer the new nokia6650 m...
5,ham,it shall be fine i have avalarr now will hol...
6,ham,i love u 2 my little pocy bell i am sorry but ...
7,ham,they finally came to fix the ceiling
8,ham,s i m watching it in live
9,spam,hi the sexychat girls are waiting for you to ...


## Create the Vocabulary

We've just cleaned the data by removing punctuations and turning each letter into lower case. Now it's time to create our vocabulary.

Recall that, with the exception of the **label** column, every other column in the transformed table in the previous paragraph represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message). It's worth mentioning that we call the set of unique words a **vocabulary**.

We'll eventually bring the training set to that format ourselves, but first, let's create a list with all of the unique words that occur in the messages of our training set.

In [10]:
# Creating a vocabulary for the messages in the training set

training_set['SMS'] = training_set['SMS'].str.split() # Transform each message into a list
 
vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

It turns out that there are 7,767 unique words in all of the messages in our training set.

In [11]:
len(vocabulary)

7767

## The Final Training Set

We managed to create the vocabulary for our messages in the training set. Now we can use this vocabulary to make the data transformation we need. Let's remind ourselves of how the change will look like with a picture:

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)

First, we'll build a dictionary that we'll then convert to a DataFrame. To do that, we will:
    
- Initialize a dictionary named `word_counts_per_sms`, where each key is a unique word (string) from the vocabulary, and each value is a list of the length of the training set, where each element in the list is a `0`.
- Loop over `training_set['SMS']` using the `enumerate()` function to get both the index and the SMS message

We're now going to use the vocabulary we just created to make the data transformation we want.

In [12]:
# Creating a dictionary for our training set

word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
# Transforming word_counts_per_sms into a DataFrame
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.iloc[:, 980:1000].head()

,ana,anal,analysis,and,anderson,andre,andrews,andros,angels,angry,animal,animation,anjie,anjola,anna,annie,anniversary,annoncement,announced,announcement
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)

## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [14]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters


Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [15]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()   # spam_messages already defined in a cell above
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()   # ham_messages already defined in a cell above
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [16]:
import re

def classify(message):
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [17]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.1224830441571638e-25
P(Ham|message): 3.0591074119588186e-27
Label: Spam


In [18]:
classify('Sounds good, Tom, then see u there.')

P(Spam|message): 3.1724940547308834e-25
P(Ham|message): 5.2108593896945375e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [19]:
def classify_test_set(message):    
    '''
    message: a string
    '''
    
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [20]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head(20)

,Label,SMS,predicted
0,ham,"Sorry da thangam, very very sorry i am held up...",ham
1,ham,I.ll post her out l8r. In class,ham
2,ham,Also remember to get dobby's bowl from your car,ham
3,ham,Thanks honey. Have a great day.,ham
4,ham,LMAO where's your fish memory when I need it?,ham
5,ham,Ok...,ham
6,ham,"I keep seeing weird shit and bein all ""woah"" t...",ham
7,ham,I'm in chennai velachery:),ham
8,ham,Wat makes u thk i'll fall down. But actually i...,ham
9,ham,Wot is u up 2 then bitch?,ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [21]:
correct = 0
total = test_set.shape[0]
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1101
Incorrect: 13
Accuracy: 0.9883303411131059


The accuracy is close to 98.83%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.

## Conclusions

In this project, we created a highly accurate spam filter based on the multinomial Naive Bayes algorithm and a dataset of labeled 5,572 SMS. The spam filter takes in a new message and classifies it as spam or ham. We managed to reach an accuracy of 98.74%, which is almost 20% higher than our initial focus. Below are some additional conclusions and insights from this project:
-	A few messages classified incorrectly have some features in common. False spam messages tend to be very short, have the words absent in the vocabulary, contain typical spam-like words, or neutral words previously detected, by coincidence, only in spam messages. False ham messages tend to be rather long and have a high percentage of neutral words or the words absent in the vocabulary. In the undefined messages, we can expect an approximately proportional mixture of neutral and spam-like words.
-	The attempt to increase the accuracy even further by making the algorithm sensitive to letter case resulted, just the opposite, in rendering the spam filter much less efficient, with the accuracy dropped by 13.5%. It seems that the letter case doesn't make any valuable difference when it comes to distinguishing between spam and ham messages. 
-	The 100 most popular meaningful spam-prone words revealed the following patterns: encouraging people to do further actions, promising them something alluring, urging them, having sexual context, inviting to visit some web resources, advertising various digital devices and products.